## Скачиваем необходимое

Сначала нужно средствами NLTK загрузить WordNet.



In [ ]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## Готовим данные к работе

Затем импортируем данные из подготовленного текстового файла. Файл содержит набор пар слов (только имён существительных), для которых известны экспертные оценки сходства.

Строим ассоциативный массив "пара слов -- оценка близости".

In [ ]:
with open("Task_4_sample_3.csv", encoding="utf-8") as rf:
  triples = [line.strip().split(",") for line in rf.readlines()]
  triples.pop(0)
  score_map = {tuple(triple[:2]): float(triple[2]) for triple in triples}

Отметим, что из исходного набора данных мы взяли только экспертные оценки сходства (similarity) и только для существительных. Исходный набор данных доступен по [ссылке](http://alfonseca.org/pubs/ws353simrel.tar.gz)

Посмотрим на примеры оценок. 

У слов может быть по несколько значений, которые различаются в WordNet. Здесь -- ради примера -- мы будем "жадно" выбирать первое попавшееся, но далее будем работать с ними иначе.




In [ ]:
# for w1, w2 in list(score_map):
  
#   print("\nWords: %s-%s\nGround truth score: %.2f" % (w1, w2, score_map[(w1, w2)]))
  
#   ss1 = wn.synset(w1 + ".n.01")
#   ss2 = wn.synset(w2 + ".n.01")

#   print("\nPath: %.3f" % ss1.path_similarity(ss2), end=" ")
#   print("\nwup: %.3f" % ss1.wup_similarity(ss2), end=" ")
#   print("\nshortest_path: %.3f" % ss1.shortest_path_distance(ss2))

По ссылке представлена выборка из датасета WordSim353. Ваша задача, используя тезаурус WordNet, вычислить оценки близости для всех элементов соответствующих синсетов представленных пар слов. В качестве близости двух слов использовать наибольшую близость среди соответствующих элементов рассматриваемых синсетов. В качестве меры близости использовать: близость на основе пути (path_similarity), меру Leacock-Chodorow (lch_similarity) и меру Wu-Palmer (wup_similarity). Вычислить коэффициент ранговой корреляции Спирмена для каждой меры близости, используя известную экспертную оценку (колонка Score в выборке).

In [ ]:
from itertools import product

list_pairs = list(score_map)
path_list, lch_list, wup_list, true_list = [], [], [], []

# для всех пар
for w1, w2 in list_pairs:

  try:
    all_w1 = wn.synsets(w1, pos="n")
    all_w2 = wn.synsets(w2, pos="n")

    # добавляем интересующие нас метрики и экспертные оценки
    path = max([item1.path_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    path_list.append(path)

    lch = max([item1.lch_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    lch_list.append(lch)

    wup = max([item1.wup_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    wup_list.append(wup)
    
    true_list.append(score_map[(w1, w2)])

  except Exception as e:
    print(w1, w2, "error:", e)

## Вычисляем ранговую корреляцию Спирмена

In [ ]:
from scipy.stats import spearmanr

coef, p = spearmanr(path_list, true_list)
print("path Spearman R: %.3f" % coef)

coef, p = spearmanr(lch_list, true_list)
print("lch Spearman R: %.3f" % coef)

coef, p = spearmanr(wup_list, true_list)
print("wup  Spearman R: %.3f" % coef)

path Spearman R: 0.680
lch Spearman R: 0.680
wup  Spearman R: 0.693


При помощи метода hyponyms() найдите количество гипонимов для синсета student.n.01, а также при помощи метода name() найдите значение первого в гипонима из списка.

Введите количество гипонимов для синсета student.n.01.

Первый элемент списка гипонимов синсета student.n.01.

In [ ]:
count = len(wn.synset('student.n.01').hyponyms())

print(count)
[str(lemma.name()) for lemma in wn.synset('student.n.01').hyponyms()]

20


['art_student.n.01',
 'auditor.n.02',
 'catechumen.n.01',
 'collegian.n.01',
 'crammer.n.01',
 'etonian.n.01',
 'ivy_leaguer.n.01',
 'law_student.n.01',
 'major.n.03',
 'medical_student.n.01',
 'nonreader.n.01',
 'overachiever.n.01',
 'passer.n.03',
 'scholar.n.03',
 'seminarian.n.01',
 'sixth-former.n.01',
 'skipper.n.01',
 'underachiever.n.01',
 'withdrawer.n.05',
 'wykehamist.n.01']